In [1]:
# ====================================================
# Library
# ====================================================
import os
import gc
import sys
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM

sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import timm

from torch.cuda.amp import autocast, GradScaler

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

/home/yamanaka/.pyenv/versions/anaconda3-2023.03/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/yamanaka/.pyenv/versions/anaconda3-2023.03/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# ====================================================
# CFG
# ====================================================
class CFG:
    apex=False
    debug=False
    print_freq=10
    num_workers=0
    size=224
    model_name='vgg16'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=40
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=32
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    # seed = [18, 24]
    # seed= [19, 23, 34, 40, 56, 77, 88, 99, 12, 23, 34, 45, 56, 67, 78, 89]
    seed= [42, 10, 20, 51, 111]
    target_size=1
    target_col='KIc'
    n_fold=5
    trn_fold=[i for i in range(n_fold)]
    kfold="Kfold" #or Kfold
    train=True
    grad_cam=False

# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './KIc/Model/vgg/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [3]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
    return score


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


In [4]:
# ====================================================
# Dataset
# ====================================================
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df[CFG.target_col].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)['image']
        label = torch.tensor(self.labels[idx]).float()
        return image, label

In [5]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):
    
    if data == 'train':
        return A.Compose([
            # A.Resize(CFG.size, CFG.size),
            A.RandomResizedCrop(CFG.size, CFG.size, scale=(0.85, 1.0)),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

    elif data == 'valid':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])


In [6]:
# ====================================================
# MODEL
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, pretrained=True):
        super().__init__()
        self.cfg = cfg
        self.model = timm.create_model(self.cfg.model_name, pretrained=pretrained)
        self.n_features = self.model.head.in_features
        self.model.head.fc = nn.Identity()
        self.fc = nn.Linear(self.n_features, self.cfg.target_size)

    def feature(self, image):
        feature = self.model(image)
        return feature
        
    def forward(self, image):
        feature = self.feature(image)
        output = self.fc(feature)
        return output

In [7]:
# ====================================================
# Loss
# ====================================================
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss

In [8]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    if CFG.apex:#使わないところは消す
        scaler = GradScaler()
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        if CFG.apex:
            with autocast():
                y_preds = model(images)
                loss = criterion(y_preds.view(-1), labels)
        else:
            y_preds = model(images)
            loss = criterion(y_preds.view(-1), labels)
        # record loss
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            if CFG.apex:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.6f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))

    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    losses = AverageMeter()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds.view(-1), labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        preds.append(y_preds.to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [9]:
# ====================================================
# Train loop
# ====================================================
def train_loop(folds, fold, seed):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_col].values

    train_dataset = TrainDataset(train_folds, transform=get_transforms(data='train'))
    valid_dataset = TrainDataset(valid_folds, transform=get_transforms(data='train'))

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size, 
                              shuffle=True, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, 
                              batch_size=CFG.batch_size * 2, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    # ====================================================
    # scheduler 
    # ====================================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
        return scheduler

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, pretrained=True)
    model.to(device)

    optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = get_scheduler(optimizer)

    # ====================================================
    # loop
    # ====================================================
    criterion = RMSELoss()

    best_score = np.inf
    best_loss = np.inf
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()
        
        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()

        # scoring
        score = get_score(valid_labels, preds)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
       
        if score < best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 'preds': preds}, OUTPUT_DIR+f'{CFG.model_name}_{CFG.kfold}_fold{fold}_seed{seed}_bootstrap.pth')
    
    valid_folds['preds'] = torch.load(OUTPUT_DIR+f'{CFG.model_name}_{CFG.kfold}_fold{fold}_seed{seed}_bootstrap.pth', 
                                      map_location=torch.device('cpu'))['preds']

    return valid_folds

In [10]:
# ====================================================
# main
# ====================================================
def main():

    """
    Prepare: 1.train 
    """

    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df[CFG.target_col].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}')
    
    for seed in CFG.seed:
        LOGGER.info(f"========== seed{seed} ==========")
        seed_torch()

        data = pd.read_csv('/home/yamanaka/Estimate_KIc_with_ViT/Mototake_Analysis/VGG+GP/inout_data.csv', header=None, names=['Id', 'KIc'])
        data['file_path'] = ['/home/yamanaka/Estimate_KIc_with_ViT/Mototake_Analysis/VGG+GP/imagedata/' + str(i) + '.jpg' for i in data['Id']]

        train = pd.DataFrame()
        data_id = np.random.choice(data['Id'], size=len(data), replace=True)
        for i in data_id:
            train = pd.concat([train, data[data['Id']==i]])
        train = train.reset_index()
            
        if CFG.debug:
            CFG.epochs = 1
            train = train.sample(n=100, random_state=seed).reset_index(drop=True)

        if CFG.kfold == 'Kfold':
            Fold = KFold(n_splits=CFG.n_fold, shuffle=True, random_state=seed)
            for n, (train_index, val_index) in enumerate(Fold.split(train)):
                train.loc[val_index, 'fold'] = int(n)
            train['fold'] = train['fold'].astype(int)
        elif CFG.kfold == "StratifiedKfold":
            num_bins = int(np.floor(1 + np.log2(len(train))))
            train["bins"] = pd.cut(train[CFG.target_col], bins=num_bins, labels=False)
            Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=seed)
            for n, (train_index, val_index) in enumerate(Fold.split(train, train["bins"])):
                train.loc[val_index, 'fold'] = int(n)
            train['fold'] = train['fold'].astype(int)

        # # train出力 vgg 
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            # train.to_csv vggも
            _oof_df = train_loop(train, fold, seed)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df)

        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)

        # save result
        oof_df.to_csv(OUTPUT_DIR+f'{CFG.model_name}_{CFG.kfold}_seed{seed}_bootstrap_oof_df.csv', index=False)

In [11]:
if __name__ == '__main__':
    main()

========== seed42 ==========
========== fold: 0 training ==========


Epoch: [1][0/19] Elapsed 0m 1s (remain 0m 26s) Loss: 3.1279(3.1279) Grad: 25.2494  LR: 0.000100  
Epoch: [1][10/19] Elapsed 0m 2s (remain 0m 2s) Loss: 0.4898(1.1408) Grad: 16.4406  LR: 0.000100  
Epoch: [1][18/19] Elapsed 0m 4s (remain 0m 0s) Loss: 0.4128(0.8990) Grad: 5.8513  LR: 0.000100  
EVAL: [0/3] Elapsed 0m 0s (remain 0m 0s) Loss: 0.6058(0.6058) 


Epoch 1 - avg_train_loss: 0.8990  avg_val_loss: 0.5912  time: 5s
Epoch 1 - Score: 0.5915
Epoch 1 - Save Best Score: 0.5915 Model


EVAL: [2/3] Elapsed 0m 0s (remain 0m 0s) Loss: 0.6133(0.5912) 
Epoch: [2][0/19] Elapsed 0m 0s (remain 0m 2s) Loss: 0.5590(0.5590) Grad: 24.8614  LR: 0.000057  
Epoch: [2][10/19] Elapsed 0m 1s (remain 0m 1s) Loss: 0.5666(0.4986) Grad: 23.4378  LR: 0.000057  
Epoch: [2][18/19] Elapsed 0m 2s (remain 0m 0s) Loss: 0.4089(0.4734) Grad: 3.9394  LR: 0.000057  
EVAL: [0/3] Elapsed 0m 0s (remain 0m 0s) Loss: 0.4856(0.4856) 


Epoch 2 - avg_train_loss: 0.4734  avg_val_loss: 0.4637  time: 3s
Epoch 2 - Score: 0.4642
Epoch 2 - Save Best Score: 0.4642 Model


...

EVAL: [2/3] Elapsed 0m 0s (remain 0m 0s) Loss: 0.1523(0.2401) 
Epoch: [39][0/19] Elapsed 0m 0s (remain 0m 2s) Loss: 0.1142(0.1142) Grad: 6.6658  LR: 0.000009  
Epoch: [39][10/19] Elapsed 0m 1s (remain 0m 1s) Loss: 0.0560(0.1094) Grad: 9.8532  LR: 0.000009  
Epoch: [39][18/19] Elapsed 0m 2s (remain 0m 0s) Loss: 0.0827(0.1003) Grad: 5.7278  LR: 0.000009  
EVAL: [0/3] Elapsed 0m 0s (remain 0m 0s) Loss: 0.2826(0.2826) 


Epoch 39 - avg_train_loss: 0.1003  avg_val_loss: 0.2234  time: 4s
Epoch 39 - Score: 0.2303


EVAL: [2/3] Elapsed 0m 0s (remain 0m 0s) Loss: 0.1307(0.2234) 
Epoch: [40][0/19] Elapsed 0m 0s (remain 0m 2s) Loss: 0.0726(0.0726) Grad: 4.6380  LR: 0.000001  
Epoch: [40][10/19] Elapsed 0m 1s (remain 0m 1s) Loss: 0.0799(0.0854) Grad: 2.5885  LR: 0.000001  
Epoch: [40][18/19] Elapsed 0m 2s (remain 0m 0s) Loss: 0.0591(0.0805) Grad: 2.6271  LR: 0.000001  
EVAL: [0/3] Elapsed 0m 0s (remain 0m 0s) Loss: 0.2970(0.2970) 


Epoch 40 - avg_train_loss: 0.0805  avg_val_loss: 0.2313  time: 4s
Epoch 40 - Score: 0.2384
========== fold: 4 result ==========
Score: 0.2247
========== CV ==========
Score: 0.2199


EVAL: [2/3] Elapsed 0m 0s (remain 0m 0s) Loss: 0.1502(0.2313) 
